# Analysis of mitigation potential and trade-offs of decarbonization strategies

**Implemented decarbonization strategies**

 S1. Decarbonization of on-site electricity consumption (switch to wind power + battery storage)

 S2. Improvements in energy efficiency (5% reduction, assuming an annual reduction of 0.5% over 10 years)

 S3. Decarbonization of process heat (electrification of heat)

 S4. Decarbonization of electricity in chemical indsutry (switch to wind power + battery storage)

 S5. Decarbonization of process heat in chemical industry (electrification of heat)

In [1]:
%run _imports.ipynb

BW_PROJECT = 'lib_rm' # insert your project name here
bw.projects.set_current(BW_PROJECT)

## Create LCI databases with implemented decarbonization strategies

In [2]:
EI_DB_NAME = 'ecoinvent-3.10-cutoff'   # name of ecoinvent database in your project
LIB_RM_DB_NAME = "LIB raw materials"   # name of databases containing LCIs for LIB raw materials

# List of LCI databases for decarbonization strategies
dbs_strategies = {
    "BAU": LIB_RM_DB_NAME,
    "ONSITE_ELEC": LIB_RM_DB_NAME + ", ONSITE_ELEC",
    "ENERGY_EFF": LIB_RM_DB_NAME + ", ONSITE_ELEC + ENERGY_EFF",
    "PROCESS_HEAT": LIB_RM_DB_NAME + ", ONSITE_ELEC + ENERGY_EFF + PROCESS_HEAT",
    "CHEMICALS_DECARB": LIB_RM_DB_NAME + ", ONSITE_ELEC + ENERGY_EFF + PROCESS_HEAT + CHEMICALS_DECARB",
    "GLOBAL_ELEC": LIB_RM_DB_NAME + ", ONSITE_ELEC + ENERGY_EFF + PROCESS_HEAT + CHEMICALS_ELEC + GLOBAL_ELEC",
}

technosphere = lambda x: x["type"] == "technosphere"
biosphere = lambda x: x["type"] == "biosphere"

breakdown_lists = supporting_functions.get_breakdown_lists()
inventories_breakdown = pd.read_excel(INVENTORIES_PATH, sheet_name="Datasets for breakdown", index_col=0, skiprows=1)

In [3]:
# Import ecoinvent database into wurst format (i.e., list of dictionary, each dict being a dataset)
try:
  len(ei_db)
except NameError:
  ei_db = wurst.extract_brightway2_databases(EI_DB_NAME)

# Import LIB raw materials inventories into wurst format (i.e., list of dictionary, each dict being a dataset)
try:
  len(lib_rm_db)
except NameError:
  lib_rm_db = wurst.extract_brightway2_databases(LIB_RM_DB_NAME)

# Import biosphere database into dictionary
biosphere_db = [ef.as_dict() for ef in bw.Database('ecoinvent-3.10-biosphere')]

Getting activity data


100%|██████████| 23523/23523 [00:00<00:00, 159366.90it/s]


Adding exchange data to activities


100%|██████████| 743409/743409 [01:31<00:00, 8166.48it/s] 


Filling out exchange data


...

100%|██████████| 31/31 [00:00<00:00, 31002.25it/s]


Adding exchange data to activities


100%|██████████| 498/498 [00:00<00:00, 12097.13it/s]


Filling out exchange data


100%|██████████| 31/31 [00:00<00:00, 55.49it/s]


In [4]:
# Datasets used to model decarbonization strategies

# On-site electricity source shifts to wind onshore + battery storage
wind_filter = {
    'name': "electricity production, wind, 1-3MW turbine, onshore",
    'product': "electricity, high voltage",
    'unit': "kilowatt hour"
    }

wind_battery_system_filter = {
    'name': "electricity consumption, wind, 3MW turbine, onshore, with LFP battery storage",
    'product': "electricity, high voltage",
    'unit': "kilowatt hour"
    }

battery_filter = {
    'name': "electricity production, wind, 3MW turbine, onshore, at LFP battery",
    'product': "electricity, high voltage",
    'unit': "kilowatt hour"
    }

# Process heat is electrified
heat_low_temp_filter = {
    'name': "heat production, at heat pump 30kW, allocation exergy",
    'product': "heat, central or small-scale, other than natural gas",
    'unit': "megajoule"
    }

heat_high_temp_filter = {
    'name': "heat production, electric furnace",
    'product': "heat, district or industrial, other than natural gas",
    'unit': "megajoule"
    }

# List of countries for LIB raw materials production processes
list_countries_LIB = list(set([ds["location"] for ds in lib_rm_db if ds["location"]]))

### 1. Decarbonization of on-site electricity consumption

In [5]:
# Copy of databases for modification:
ONSITE_ELEC_db = copy.deepcopy(lib_rm_db)

In [6]:
# Create country markets for wind electricity generation for countries 
# with higher regionalization
countries = ["CN", "CA", "IN", "US"]
wind_elec_country_markets = supporting_functions.create_renewable_electricity_country_market(
    wind_filter["name"], countries, ei_db, dbs_strategies["ONSITE_ELEC"])

# Add new markets to the database:
ONSITE_ELEC_db.extend(wind_elec_country_markets)

# Create a dictionary with wind electricity dataset for each location in ecoinvent
wind_elec_ds_locs = {}
ei_locs = list(set([ds["location"] for ds in ei_db]))

for loc in ei_locs:
    if loc not in wind_elec_ds_locs:

        if loc in countries:
            wind_filter_market = copy.deepcopy(wind_filter)
            wind_filter_market.update({
                "name": "market for " + wind_filter["name"]
            })
            wind_ds = supporting_functions.get_dataset_for_location(wind_filter_market, loc, wind_elec_country_markets)
        else:
            try:
                wind_ds = supporting_functions.get_dataset_for_location(wind_filter, loc, ei_db)
            except KeyError:
                wind_ds = supporting_functions.get_dataset_for_location(wind_filter, "RoW", ei_db)
            
        wind_elec_ds_locs[loc] = wind_ds

RoE not found in ISO3
CN-SWG not found in regex
AU-WA not found in regex
AU-NSW not found in regex
AU-SA not found in regex
CN-CCG not found in regex
CN-ECGC not found in regex
AU-VIC not found in regex
Europe,  not found in regex
AU-QLD not found in regex
AU-TAS not found in regex
CN-NWG not found in regex
CN-NECG not found in regex
BR-South-eastern/Mid-western grid not found in regex
CN-NCGC not found in regex


In [7]:
# Regionalize wind-battery system and battery datasets

# Get wind-battery system and battery datasets
wind_battery_system_ds = supporting_functions.get_dataset_for_location(wind_battery_system_filter, "GLO", ONSITE_ELEC_db)
battery_ds = supporting_functions.get_dataset_for_location(battery_filter, "GLO", ONSITE_ELEC_db)

for loc in list_countries_LIB:

    if loc == "GLO":
        # GLO datasets already exist
        pass
    else:
        # Replicate wind-battery system and battery datasets to location
        wind_battery_system_loc = supporting_functions.replicate_activity_to_location(wind_battery_system_ds, loc, ei_db + ONSITE_ELEC_db, dbs_strategies["ONSITE_ELEC"])
        battery_loc = supporting_functions.replicate_activity_to_location(battery_ds, loc, ei_db + ONSITE_ELEC_db, dbs_strategies["ONSITE_ELEC"])

        #... and substitute wind power and battery input in that location
        for ds in [wind_battery_system_loc, battery_loc]:
            for exc in filter(technosphere, ds["exchanges"]):
                if exc['name'] == "electricity production, wind, 1-3MW turbine, onshore":
                    exc.update({
                            'name': wind_elec_ds_locs[loc]['name'],
                            'product': wind_elec_ds_locs[loc]['reference product'],
                            'location': wind_elec_ds_locs[loc]['location'],
                            'input': (wind_elec_ds_locs[loc]['database'], wind_elec_ds_locs[loc]['code'])}
                        )

                elif exc['name'] == "electricity production, wind, 3MW turbine, onshore, at LFP battery":
                    exc.update({
                        'name': battery_loc['name'],
                        'product': battery_loc['reference product'],
                        'location': battery_loc['location'],
                        'input': (battery_loc['database'], battery_loc['code'])}
                        )

            ONSITE_ELEC_db.append(ds)

In [8]:
# Switch LIB raw materials proudction processes to wind power coupled with battery storage

# Get wind-batery system dataset for each location with LIB raw materials production:
wind_battery_ds_locs = {}
for loc in list_countries_LIB:
    wind_battery_ds_locs[loc] = [a for a in ONSITE_ELEC_db if a['name'] == wind_battery_system_filter["name"] and a["location"] == loc]

# Switch to wind power coupled with battery storage
for ds in ONSITE_ELEC_db:
    # Only for LIB raw materials production processes
    if ds["name"] in list(inventories_breakdown["name"]):

        # Replace electricity with wind-battery system in the activities
        for exc in filter(technosphere, ds["exchanges"]):
            if exc['product'] in ['electricity, low voltage', "electricity, medium voltage", "electricity, high voltage"]:
                exc.update({
                    'name': wind_battery_ds_locs[exc["location"]][0]['name'],
                    'product': wind_battery_ds_locs[exc["location"]][0]['reference product'],
                    'location': wind_battery_ds_locs[exc["location"]][0]['location'],
                    'input': (wind_battery_ds_locs[exc["location"]][0]['database'], wind_battery_ds_locs[exc["location"]][0]['code'])})

In [19]:
if dbs_strategies["ONSITE_ELEC"] in bw.databases:
    del bw.databases[dbs_strategies["ONSITE_ELEC"]]
wurst.write_brightway2_database(ONSITE_ELEC_db, dbs_strategies["ONSITE_ELEC"])

Writing activities to SQLite3 database:


45 datasets
646 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/31/2024 00:00:22
  Finished: 07/31/2024 00:00:22
  Total time elapsed: 00:00:00
  CPU %: 19.90
  Memory %: 36.90
Created database: LIB raw materials, ONSITE_ELEC


### 2. Improvements in energy efficiency

In [9]:
# Copy of databases for modification:
ENERGY_EFF_db = copy.deepcopy(ONSITE_ELEC_db)

# 5% improvement in energy efficiency
ENERGY_EFF_IMPROV = 0.05

In [10]:
# Reduce electricity and heat consumption by the energy efficiency improvement
for ds in ENERGY_EFF_db:
    
    # Energy efficiency improvements do not apply to the wind-battery system and battery dataset:
    if ds["name"] in ["electricity consumption, wind, 3MW turbine, onshore, with LFP battery storage",
                      "electricity production, wind, 3MW turbine, onshore, at LFP battery"]:
      pass
    
    else:
      for exc in filter(technosphere, ds["exchanges"]):
              if exc['product'] in breakdown_lists["electricity products"] + \
                                  breakdown_lists["heating products"] + \
                                  ["diesel, burned in building machine", "machine operation, diesel, >= 74.57 kW, high load factor"]:  
                    exc.update({
                      'amount': exc["amount"] * (1 - ENERGY_EFF_IMPROV)
                    })

In [24]:
if dbs_strategies["ENERGY_EFF"] in bw.databases:
    del bw.databases[dbs_strategies["ENERGY_EFF"]]
wurst.write_brightway2_database(ENERGY_EFF_db, dbs_strategies["ENERGY_EFF"])

Writing activities to SQLite3 database:


45 datasets
646 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/31/2024 00:01:41
  Finished: 07/31/2024 00:01:41
  Total time elapsed: 00:00:00
  CPU %: 87.50
  Memory %: 36.92
Created database: LIB raw materials, ONSITE_ELEC + ENERGY_EFF


### 3. Electrification of process heat

In [82]:
# Copy of databases for modification:
PROCESS_HEAT_db = copy.deepcopy(ENERGY_EFF_db)

In [83]:
# Create regionalized heat pump and electric furnace datasets:

# Get heat pump and electric furnace dataset
heat_pump_ds = supporting_functions.get_dataset_for_location(heat_low_temp_filter, "RoW", ei_db)
elec_furnace_ds = supporting_functions.get_dataset_for_location(heat_high_temp_filter, "GLO", PROCESS_HEAT_db)

for loc in list_countries_LIB:
    # Create heat pump and electric furnace datasets for location 
    heat_pump_loc = supporting_functions.replicate_activity_to_location(heat_pump_ds, loc, ei_db + PROCESS_HEAT_db, dbs_strategies["PROCESS_HEAT"])
    elec_furnace_loc = supporting_functions.replicate_activity_to_location(elec_furnace_ds, loc, ei_db + PROCESS_HEAT_db, dbs_strategies["PROCESS_HEAT"])

    if loc == "GLO":
        # Electric furnace already exists for "GLO"
        ds_to_chage = [heat_pump_loc]
    else:
        ds_to_chage = [heat_pump_loc, elec_furnace_loc]
    
    # Replace electricity with wind-battery system in the activities
    for ds in ds_to_chage:
        for exc in filter(technosphere, ds["exchanges"]):
            if exc['product'] in breakdown_lists["electricity products"]:
                exc.update({
                    'name': wind_battery_ds_locs[ds["location"]][0]['name'],
                    'product': wind_battery_ds_locs[ds["location"]][0]['reference product'],
                    'location': wind_battery_ds_locs[ds["location"]][0]['location'],
                    'input': (dbs_strategies["PROCESS_HEAT"], wind_battery_ds_locs[ds["location"]][0]['code'])})
            
        PROCESS_HEAT_db.append(ds)

# Replace electricity with wind-battery system in the the
# existing GLO electric furnace dataset
for ds in PROCESS_HEAT_db:
    if ds["name"] == elec_furnace_ds["name"] and ds["location"] == "GLO":
        for exc in filter(technosphere, ds["exchanges"]):
            if exc['product'] in breakdown_lists["electricity products"]:
                exc.update({
                    'name': wind_battery_ds_locs[ds["location"]][0]['name'],
                    'product': wind_battery_ds_locs[ds["location"]][0]['reference product'],
                    'location': wind_battery_ds_locs[ds["location"]][0]['location'],
                    'input': (dbs_strategies["PROCESS_HEAT"], wind_battery_ds_locs[ds["location"]][0]['code'])})

In [84]:
[ds for ds in PROCESS_HEAT_db if ds["name"]==heat_pump_loc["name"] and ds["location"] == "GLO"]

[{'classifications': [('EcoSpold01Categories', 'heat pumps/heating systems'),
   ('ISIC rev.4 ecoinvent', '3530:Steam and air conditioning supply'),
   ('CPC', '17300: Steam and hot water')],
  'comment': 'Heat from 30 kW heat pump installed at 160kW cogeneration unit.\n[This dataset was already contained in the ecoinvent database version 2. It was not individually updated during the transfer to ecoinvent version 3. Life Cycle Impact Assessment results may still have changed, as they are affected by changes in the supply chain, i.e. in other datasets. This dataset was generated following the ecoinvent quality guidelines for version 2. It may have been subject to central changes described in the ecoinvent version 3 change report (http://www.ecoinvent.org/database/ecoinvent-version-3/reports-of-changes/), and the results of the central updates were reviewed extensively. The changes added e.g. consistent water flows and other information throughout the database. The documentation of this 

In [85]:
# Substitute heat sources in LIB raw material datasets:

low_temperature_ds = [
    "lithium brine production, from evaporation pond, Salar de Atacama",
    "lithium brine purification, Salar de Atacama",
    "lithium carbonate production, Salar de Atacama",
    "lithium hydroxide production, Salar de Atacama"
]

for ds in PROCESS_HEAT_db:
    if ds["name"] in list(inventories_breakdown["name"]):

        # Low-temperature heat is supplied by heat pumps
        if ds["name"] in low_temperature_ds:
            electrified_heat_ds = supporting_functions.get_dataset_for_location(heat_low_temp_filter, ds["location"], PROCESS_HEAT_db)
        else:
            electrified_heat_ds = supporting_functions.get_dataset_for_location(heat_high_temp_filter, ds["location"], PROCESS_HEAT_db)

        for exc in filter(technosphere, ds["exchanges"]):
            if exc['product'] in breakdown_lists["heating products"]:
                exc.update({
                    'name': electrified_heat_ds['name'],
                    'product': electrified_heat_ds['reference product'],
                    'location': electrified_heat_ds['location'],
                    "database": electrified_heat_ds['database'],
                    'input': (electrified_heat_ds['database'], electrified_heat_ds['code'])})

In [86]:
if dbs_strategies["PROCESS_HEAT"] in bw.databases:
    del bw.databases[dbs_strategies["PROCESS_HEAT"]]
wurst.write_brightway2_database(PROCESS_HEAT_db, dbs_strategies["PROCESS_HEAT"])

Writing activities to SQLite3 database:


56 datasets
680 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/31/2024 09:33:10
  Finished: 07/31/2024 09:33:10
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 14.70
Created database: LIB raw materials, ONSITE_ELEC + ENERGY_EFF + PROCESS_HEAT


### 4. Decarbonization of electricity and heat in reagents production

In [104]:
# Copy of databases for modification:
CHEMICALS_DECARB_db = copy.deepcopy(PROCESS_HEAT_db)
ei_CHEMICALS_DECARB_db = copy.deepcopy(ei_db)

ei_CHEMICALS_DECARB_db_name = EI_DB_NAME + " + CHEMICALS_DECARB"

In [105]:
# Since some of the chemicals used in LIB raw materials rely on
# other chemicals produced upstream, we need to ensure that
# all upstreams processes also switch to wind power and electrified heat. 
# Therefore, we change all the datasets under these categories:

chemicals_ISIC_categories = [
    '1920:Manufacture of refined petroleum products',
    '1040:Manufacture of vegetable and animal oils and fats',
    '2394:Manufacture of cement, lime and plaster',
    '0891:Mining of chemical and fertilizer minerals',
    '2011:Manufacture of basic chemicals',
    '0810:Quarrying of stone, sand and clay',
    '2012:Manufacture of fertilizers and nitrogen compounds',
    '2030:Manufacture of man-made fibres',
    '2013:Manufacture of plastics and synthetic rubber in primary forms',
    '2100:Manufacture of pharmaceuticals, medicinal chemical and botanical products',
    '2029:Manufacture of other chemical products n.e.c.',
    '2220:Manufacture of plastics products',
    '201:Manufacture of basic chemicals, fertilizers and nitrogen compounds, plastics',
    '20:Manufacture of chemicals and chemical products',
    '2023:Manufacture of soap and detergents, cleaning and polishing preparations, pe'
]

In [106]:
# Create an ecoinvent database where reagents production is 
# powered by wind-battery system and electrified heat
elec_furnace_GLO = supporting_functions.get_dataset_for_location(heat_high_temp_filter, "GLO", lib_rm_db)
wind_battery_system_GLO = supporting_functions.get_dataset_for_location(wind_battery_system_filter, "GLO", lib_rm_db)
battery_GLO = supporting_functions.get_dataset_for_location(battery_filter, "GLO", lib_rm_db)

for ds in ei_CHEMICALS_DECARB_db:
    product_category = [i[1] for i in ds["classifications"] if i[0] == 'ISIC rev.4 ecoinvent']
    
    if len(product_category) > 0:
        if product_category[0] in chemicals_ISIC_categories:

            for exc in filter(technosphere, ds["exchanges"]):

                # Switch input electricity to wind-battery system (using GLO dataset)
                if exc['product'] in breakdown_lists["electricity products"]:
                    exc.update({
                            'name': wind_battery_system_GLO['name'],
                            'product': wind_battery_system_GLO['reference product'],
                            'location': wind_battery_system_GLO['location'],
                            'input': (dbs_strategies["PROCESS_HEAT"], wind_battery_system_GLO['code'])}
                        )
                
                # Switch input process heat to electric furnace powereed by wind-battery system (using GLO dataset)
                elif exc['product'] in breakdown_lists["heating products"]:
                    exc.update({
                        'name': elec_furnace_ds['name'],
                        'product': elec_furnace_ds['reference product'],
                        'location': elec_furnace_ds['location'],
                        "database": elec_furnace_ds['database'],
                        'input': (dbs_strategies["PROCESS_HEAT"], elec_furnace_ds['code'])}
                    )

In [107]:
# Write the modified ecoinvent database to memory
if ei_CHEMICALS_DECARB_db_name in bw.databases:
    del bw.databases[ei_CHEMICALS_DECARB_db_name]
wurst.write_brightway2_database(ei_CHEMICALS_DECARB_db, ei_CHEMICALS_DECARB_db_name)

Vacuuming database 
23523 datasets
743409 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:52


Title: Writing activities to SQLite3 database:
  Started: 07/31/2024 10:14:17
  Finished: 07/31/2024 10:15:10
  Total time elapsed: 00:00:52
  CPU %: 31.20
  Memory %: 32.66
Created database: ecoinvent-3.10-cutoff + CHEMICALS_DECARB


In [108]:
# Link raw materials datasets to the modified ecoinvent version
for ds in CHEMICALS_DECARB_db:
    for exc in filter(technosphere, ds["exchanges"]):
        exc_ds = [
            d for d in ei_CHEMICALS_DECARB_db if 
            d["name"] == exc["name"] and
            d["reference product"] == exc["product"] and
            d["location"] == exc["location"] and
            d["unit"] == exc["unit"]
        ]
        if len(exc_ds) > 0:
            exc.update({
                "input": (ei_CHEMICALS_DECARB_db_name, exc_ds[0]["code"])
                })

In [109]:
if dbs_strategies["CHEMICALS_DECARB"] in bw.databases:
    del bw.databases[dbs_strategies["CHEMICALS_DECARB"]]
wurst.write_brightway2_database(CHEMICALS_DECARB_db, dbs_strategies["CHEMICALS_DECARB"])

Writing activities to SQLite3 database:


56 datasets
680 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/31/2024 10:19:59
  Finished: 07/31/2024 10:19:59
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 30.78
Created database: LIB raw materials, ONSITE_ELEC + ENERGY_EFF + PROCESS_HEAT + CHEMICALS_DECARB


### 5. Wind power in the global economy

In [110]:
# Copy of databases for modification:
GLOBAL_ELEC_db = copy.deepcopy(CHEMICALS_DECARB_db)
ei_GLOBAL_ELEC_db = copy.deepcopy(ei_CHEMICALS_DECARB_db)

ei_GLOBAL_ELEC_db_name = ei_CHEMICALS_DECARB_db_name + " + GLOBAL_ELEC"

In [111]:
# Create an ecoinvent database where all activities are powered by wind-battery system
# considering the GLO dataset
for ds in ei_GLOBAL_ELEC_db:
    for exc in filter(technosphere, ds["exchanges"]):
        if exc['product'] in breakdown_lists["electricity products"]:

            exc.update({
                'name': wind_battery_ds_locs["GLO"][0]['name'],
                'product': wind_battery_ds_locs["GLO"][0]['reference product'],
                'location': wind_battery_ds_locs["GLO"][0]['location'],
                'input': (dbs_strategies["PROCESS_HEAT"], wind_battery_ds_locs["GLO"][0]['code'])}
            )

In [112]:
# Write the modified ecoinvent database to memory
if ei_GLOBAL_ELEC_db_name in bw.databases:
    del bw.databases[ei_GLOBAL_ELEC_db_name]
wurst.write_brightway2_database(ei_GLOBAL_ELEC_db, ei_GLOBAL_ELEC_db_name)

Vacuuming database 
23523 datasets
743409 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:03


Title: Writing activities to SQLite3 database:
  Started: 07/31/2024 10:24:46
  Finished: 07/31/2024 10:25:50
  Total time elapsed: 00:01:03
  CPU %: 39.90
  Memory %: 32.66
Created database: ecoinvent-3.10-cutoff + CHEMICALS_DECARB + GLOBAL_ELEC


In [113]:
# Link raw materials datasets to the modified ecoinvent version
for ds in GLOBAL_ELEC_db:
    for exc in filter(technosphere, ds["exchanges"]):
        exc_ds = [
            d for d in ei_GLOBAL_ELEC_db if 
            d["name"] == exc["name"] and
            d["reference product"] == exc["product"] and
            d["location"] == exc["location"] and
            d["unit"] == exc["unit"]
        ]
        if len(exc_ds) > 0:
            exc.update({
                "input": (ei_GLOBAL_ELEC_db_name, exc_ds[0]["code"])
                })

In [114]:
if dbs_strategies["GLOBAL_ELEC"] in bw.databases:
    del bw.databases[dbs_strategies["GLOBAL_ELEC"]]
wurst.write_brightway2_database(GLOBAL_ELEC_db, dbs_strategies["GLOBAL_ELEC"])

Writing activities to SQLite3 database:


56 datasets
680 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/31/2024 10:29:45
  Finished: 07/31/2024 10:29:45
  Total time elapsed: 00:00:00
  CPU %: 6.60
  Memory %: 32.75
Created database: LIB raw materials, ONSITE_ELEC + ENERGY_EFF + PROCESS_HEAT + CHEMICALS_ELEC + GLOBAL_ELEC


## Calculate impacts

In [136]:
INVENTORIES = {
    "Lithium, brine":        ("lithium hydroxide production, Salar de Atacama", "lithium hydroxide, battery grade", "CL"),
    "Lithium, spodumene":    ("lithium hydroxide production, from Australian spodumene", "lithium hydroxide, battery grade", "CN"),
    "Cobalt":                ("cobalt sulfate production, from copper-cobalt ore, economic allocation", "cobalt sulfate", "CN"),
    "Nickel":                ("nickel sulfate production, average excluding China, economic allocation", "nickel sulfate", "GLO"),
    "Graphite, natural":     ("natural graphite production, battery grade, from Heilongjiang", "natural graphite, battery grade", "CN"),
    "Graphite, synthetic":   ("graphite powder coating", "synthetic graphite, battery grade", "CN"),
}

EF_METHODS = {
                'Climate change':                      ('IPCC 2021', 'climate change: including SLCFs', 'global warming potential (GWP100)'),
                'Acidification':                       ('EF v3.1', 'acidification', 'accumulated exceedance (AE)'),
                'Eutrophication, freshwater':          ('EF v3.1', 'eutrophication: freshwater', 'fraction of nutrients reaching freshwater end compartment (P)'),
                'Eutrophication, marine':              ('EF v3.1', 'eutrophication: marine', 'fraction of nutrients reaching marine end compartment (N)'),
                'Eutrophication, terrestrial':         ('EF v3.1', 'eutrophication: terrestrial', 'accumulated exceedance (AE)'),
                'Photochemical oxidant formation':     ('EF v3.1', 'photochemical oxidant formation: human health', 'tropospheric ozone concentration increase'),
                'Particulate matter formation':        ('EF v3.1', 'particulate matter formation', 'impact on human health'),
                'Ozone depletion':                     ('EF v3.1', 'ozone depletion', 'ozone depletion potential (ODP)'),
                'Ecotoxicity, freshwater':             ('EF v3.1', 'ecotoxicity: freshwater', 'comparative toxic unit for ecosystems (CTUe)'),
                'Human toxicity, carcinogenic':        ('EF v3.1', 'human toxicity: carcinogenic', 'comparative toxic unit for human (CTUh)'),
                'Human toxicity, non-carcinogenic':    ('EF v3.1', 'human toxicity: non-carcinogenic', 'comparative toxic unit for human (CTUh)'),
                'Ionising radiation':                  ('EF v3.1', 'ionising radiation: human health', 'human exposure efficiency relative to u235'), 
                'Energy resources, non-renewable':     ('EF v3.1', 'energy resources: non-renewable', 'abiotic depletion potential (ADP): fossil fuels'),
                'Material resources, minerals/metals': ('EF v3.1', 'material resources: metals/minerals', 'abiotic depletion potential (ADP): elements (ultimate reserves)'),
                'Land use':                            ('EF v3.1', 'land use', 'soil quality index'),
                'Water use':                           ('EF v3.1', 'water use', 'user deprivation potential (deprivation-weighted water consumption)')
               }



In [137]:
impacts_decarbonization = {
        db: {
            rm: 0 
            for rm in INVENTORIES.keys()} 
            for db in list(dbs_strategies.keys())
        }

In [138]:
for db in dbs_strategies:
    lca = supporting_functions.init_simple_lca(bw.Database(dbs_strategies[db]).random())

    for rm in INVENTORIES:
        rm_ds = [ds for ds in bw.Database(dbs_strategies[db])
                    if ds['name'] == INVENTORIES[rm][0] 
                    and ds['reference product'] == INVENTORIES[rm][1]
                    and ds['location'] == INVENTORIES[rm][2]][0]
        impacts = supporting_functions.multi_lcia(lca, rm_ds, EF_METHODS)
        impacts_decarbonization[db][rm] = impacts

In [139]:
impacts_decarbonization_df = pd.DataFrame(
    [
        (scenario, rm, impact, value)
        for scenario, rm_dict in impacts_decarbonization.items()
        for rm, impact_dict in rm_dict.items()
        for impact, value in impact_dict.items()
    ],
    columns=['Scenario', 'Raw material', 'Impact category', 'Value']
).set_index("Scenario")

impacts_decarbonization_df = impacts_decarbonization_df.pivot_table(index=['Raw material', 'Impact category'], columns='Scenario', values='Value').reset_index()
impacts_decarbonization_df = impacts_decarbonization_df[['Raw material', "Impact category"] + list(dbs_strategies.keys())]

impacts_decarbonization_df['Raw material'] = pd.Categorical(impacts_decarbonization_df['Raw material'], categories=INVENTORIES.keys(), ordered=True)
impacts_decarbonization_df = impacts_decarbonization_df.sort_values('Raw material').reset_index(drop=True)

In [140]:
# Export mitigation potential resutls
impacts_decarbonization_df.to_csv(DATA_DIR / "results" / f"fig4_impacts_decarbonization_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index=False)

### Monte Carlo simulation

Results are calculated only for the current situation and the decarbonization scenario with all strategies implemented

In [141]:
dbs_for_mc = ["BAU", "GLOBAL_ELEC"]
method_for_mc = {key: value for key, value in EF_METHODS.items() if key == "Climate change"}

mc_iterations = 1000

mc_results = {
        db: {
            rm: 0 
            for rm in INVENTORIES.keys()} 
            for db in dbs_for_mc
        }

In [142]:
for db in ["BAU", "GLOBAL_ELEC"]:
    print("##########", db, "##########")
    mc_lca = supporting_functions.init_mc_lca(bw.Database(dbs_strategies[db]).random())

    for rm in INVENTORIES:
        print(rm)
        rm_ds = [ds for ds in bw.Database(dbs_strategies[db])
                        if ds['name'] == INVENTORIES[rm][0] 
                        and ds['reference product'] == INVENTORIES[rm][1]
                        and ds['location'] == INVENTORIES[rm][2]][0]
        
        mc_impacts = supporting_functions.multi_lcia_MonteCarlo(mc_lca, rm_ds, mc_iterations, method_for_mc)
        mc_results[db][rm] = mc_impacts

########## BAU ##########
Lithium, brine
Lithium, spodumene
Cobalt
Nickel
Graphite, natural
Graphite, synthetic
########## GLOBAL_ELEC ##########
Lithium, brine
Lithium, spodumene
Cobalt
Nickel
Graphite, natural
Graphite, synthetic


In [143]:
mc_results_df = []
for scenario, rm_dict in mc_results.items():
    for rm, impact_dict in rm_dict.items():
        for impact, values in impact_dict.items():
            row = [scenario, rm, impact] + values.tolist()
            mc_results_df.append(row)

# Define column names
columns = ['Scenario', 'Raw material', 'Impact category'] + [i for i in range(len(values))]

# Create DataFrame
mc_results_df = pd.DataFrame(mc_results_df, columns=columns)

In [144]:
# Export Monte Carlo simulation results
mc_results_df.to_csv(DATA_DIR / "results" / f"fig4_MonteCarlo_results_iterations_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index=False)